# LangGraph Demo

LangGraph builds stateful, graph-based AI workflows where state flows through nodes connected by edges.

## Required Environment Variables

```
OPENAI_API_KEY
```

In [ ]:
# pip install langgraph langchain-openai python-dotenv

import os
from typing import List, Literal, TypedDict, Annotated
from dotenv import load_dotenv

load_dotenv(override=True)

from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from IPython.display import Markdown, display

llm = ChatOpenAI(model="gpt-4o-mini")

## State Management

State is the cornerstone of LangGraph - persistent, shared, and evolves through the workflow.

In [ ]:
class TicketState(TypedDict):
    """State schema for customer support ticket routing"""
    ticket_id: str
    title: str
    description: str
    customer_tier: str
    category: str
    priority: str
    assigned_agent: str
    processing_steps: Annotated[List[str], lambda x, y: x + y]  # List reducer
    status: str
    resolution: str

# Example state
sample_state = TicketState(
    ticket_id="TKT-12345",
    title="Login Issues with Mobile App",
    description="Customer unable to log in using mobile app",
    customer_tier="Premium",
    category="",
    priority="",
    assigned_agent="",
    processing_steps=[],
    status="New",
    resolution=""
)

print(sample_state)

## Nodes & Edges

Nodes process state, edges control flow. Each node receives the current state and returns updates.

In [ ]:
def classify_ticket(state: TicketState) -> dict:
    """Classify ticket into category using LLM"""
    print(f"Classifying: {state['ticket_id']}")

    response = llm.invoke([
        SystemMessage(content="You are a ticket classification expert."),
        HumanMessage(content=f"""
            Classify this ticket into: Technical, Billing, or General.
            Title: {state['title']}
            Description: {state['description']}
            Respond with ONLY the category name.
        """)
    ])

    category = response.content.strip()
    if category not in ["Technical", "Billing", "General"]:
        category = "General"

    return {
        "category": category,
        "processing_steps": [f"Classified as {category}"],
        "status": "Classified"
    }

def route_priority(state: TicketState) -> dict:
    """Determine ticket priority using LLM"""
    print(f"Setting priority: {state['ticket_id']}")

    response = llm.invoke([
        SystemMessage(content="You are a support ticket priority expert."),
        HumanMessage(content=f"""
            Determine priority (High, Medium, or Low) for:
            Category: {state['category']}
            Customer Tier: {state['customer_tier']}
            Title: {state['title']}
            Respond with ONLY the priority level.
        """)
    ])

    priority = response.content.strip()
    if priority not in ["High", "Medium", "Low"]:
        priority = "Medium"

    return {
        "priority": priority,
        "processing_steps": [f"Priority set to {priority}"],
        "status": "Prioritized"
    }

def match_agent(state: TicketState) -> dict:
    """Assign appropriate agent using LLM"""
    print(f"Assigning agent: {state['ticket_id']}")

    response = llm.invoke([
        SystemMessage(content="You match tickets to agents."),
        HumanMessage(content=f"""
            Match agent for: {state['category']} issue, {state['priority']} priority
            Options: Senior Tech Support, Tech Support, Billing Specialist, General Support
            Respond with ONLY the agent role name.
        """)
    ])

    agent = response.content.strip()
    valid_agents = ["Senior Tech Support", "Tech Support", "Billing Specialist", "General Support"]
    if agent not in valid_agents:
        agent = "General Support"

    return {
        "assigned_agent": agent,
        "processing_steps": [f"Assigned to {agent}"],
        "status": "Assigned"
    }

def generate_resolution(state: TicketState) -> dict:
    """Generate resolution using LLM"""
    print(f"Resolving: {state['ticket_id']}")

    response = llm.invoke([
        SystemMessage(content="You are a support resolution specialist."),
        HumanMessage(content=f"""
            Generate a brief resolution (max 30 words) for:
            Ticket: {state['title']}
            Category: {state['category']}
            Agent: {state['assigned_agent']}
        """)
    ])

    return {
        "resolution": response.content.strip(),
        "processing_steps": ["Resolution generated"],
        "status": "Resolved"
    }

## Build & Visualize Graph

In [ ]:
workflow = StateGraph(TicketState)

# Add nodes
workflow.add_node("classify", classify_ticket)
workflow.add_node("prioritize", route_priority)
workflow.add_node("assign", match_agent)
workflow.add_node("resolve", generate_resolution)

# Add edges
workflow.add_edge(START, "classify")
workflow.add_edge("classify", "prioritize")
workflow.add_edge("prioritize", "assign")
workflow.add_edge("assign", "resolve")
workflow.add_edge("resolve", END)

app = workflow.compile()

# Visualize
mermaid_text = app.get_graph().draw_mermaid()
lines = mermaid_text.split('\n')
start_idx = next(i for i, line in enumerate(lines) if 'graph' in line.lower())
display(Markdown(f"```mermaid\n{'\n'.join(lines[start_idx:])}\n```"))

## Execute Workflow

In [ ]:
test_ticket = {
    "ticket_id": "TKT-001",
    "title": "Mobile app crashes on startup",
    "description": "The iOS app crashes immediately after opening",
    "customer_tier": "Premium",
    "category": "",
    "priority": "",
    "assigned_agent": "",
    "processing_steps": [],
    "status": "New",
    "resolution": ""
}

result = app.invoke(test_ticket)

print(f"\nResult:")
print(f"  Category: {result['category']}")
print(f"  Priority: {result['priority']}")
print(f"  Agent: {result['assigned_agent']}")
print(f"  Resolution: {result['resolution']}")
print(f"  Steps: {result['processing_steps']}")

## Conditional Edges

Conditional edges enable dynamic routing based on state values.

In [ ]:
class EnhancedTicketState(TypedDict):
    """State with conditional routing fields"""
    ticket_id: str
    title: str
    description: str
    customer_tier: str
    category: str
    priority: str
    assigned_agent: str
    needs_escalation: bool
    processing_steps: Annotated[List[str], lambda x, y: x + y]
    status: str
    resolution: str

llm_senior = ChatOpenAI(model="gpt-4o")  # More capable for escalated tickets

def escalation_check(state: EnhancedTicketState) -> dict:
    """Determine if ticket needs escalation"""
    print(f"Checking escalation: {state['ticket_id']}")

    response = llm.invoke([
        SystemMessage(content="You determine if tickets need senior attention."),
        HumanMessage(content=f"""
            Should this be escalated? (YES/NO)
            Priority: {state['priority']}, Customer: {state['customer_tier']}
            Title: {state['title']}
            Escalate if: High priority, Premium+Technical, or mentions critical/urgent/outage.
        """)
    ])

    needs_escalation = response.content.strip().upper() == "YES"
    return {
        "needs_escalation": needs_escalation,
        "processing_steps": [f"Escalation: {'Yes' if needs_escalation else 'No'}"],
        "status": "Escalation Checked"
    }

def senior_agent_resolution(state: EnhancedTicketState) -> dict:
    """Senior agent handles escalated tickets with GPT-4o"""
    print(f"Senior agent handling: {state['ticket_id']}")

    response = llm_senior.invoke([
        SystemMessage(content="You are a senior technical expert."),
        HumanMessage(content=f"""
            Provide detailed resolution (50 words) for escalated ticket:
            Title: {state['title']}
            Category: {state['category']}
        """)
    ])

    return {
        "assigned_agent": "Senior Agent (GPT-4o)",
        "resolution": response.content.strip(),
        "processing_steps": ["Resolved by Senior Agent"],
        "status": "Resolved"
    }

def normal_agent_resolution(state: EnhancedTicketState) -> dict:
    """Normal agent handles standard tickets with GPT-4o-mini"""
    print(f"Normal agent handling: {state['ticket_id']}")

    response = llm.invoke([
        SystemMessage(content="You are a support agent."),
        HumanMessage(content=f"""
            Provide brief resolution (25 words) for:
            Title: {state['title']}
            Category: {state['category']}
        """)
    ])

    return {
        "assigned_agent": "Normal Agent (GPT-4o-mini)",
        "resolution": response.content.strip(),
        "processing_steps": ["Resolved by Normal Agent"],
        "status": "Resolved"
    }

def route_after_escalation(state: EnhancedTicketState) -> Literal["senior_agent", "normal_agent"]:
    """Route based on escalation decision"""
    return "senior_agent" if state['needs_escalation'] else "normal_agent"

In [ ]:
conditional_workflow = StateGraph(EnhancedTicketState)

conditional_workflow.add_node("classify", classify_ticket)
conditional_workflow.add_node("prioritize", route_priority)
conditional_workflow.add_node("escalation_check", escalation_check)
conditional_workflow.add_node("senior_agent", senior_agent_resolution)
conditional_workflow.add_node("normal_agent", normal_agent_resolution)

conditional_workflow.add_edge(START, "classify")
conditional_workflow.add_edge("classify", "prioritize")
conditional_workflow.add_edge("prioritize", "escalation_check")
conditional_workflow.add_conditional_edges(
    "escalation_check",
    route_after_escalation,
    {"senior_agent": "senior_agent", "normal_agent": "normal_agent"}
)
conditional_workflow.add_edge("senior_agent", END)
conditional_workflow.add_edge("normal_agent", END)

conditional_app = conditional_workflow.compile()

# Visualize
mermaid_text = conditional_app.get_graph().draw_mermaid()
lines = mermaid_text.split('\n')
start_idx = next(i for i, line in enumerate(lines) if 'graph' in line.lower())
display(Markdown(f"```mermaid\n{'\n'.join(lines[start_idx:])}\n```"))

In [ ]:
# Scenario 1: Critical issue -> Senior Agent
critical_ticket = {
    "ticket_id": "TKT-CRITICAL",
    "title": "URGENT: Production database down",
    "description": "Critical system failure, complete outage",
    "customer_tier": "Premium",
    "category": "", "priority": "", "assigned_agent": "",
    "needs_escalation": False, "processing_steps": [],
    "status": "New", "resolution": ""
}

print("Scenario 1: Critical Production Outage")
result1 = conditional_app.invoke(critical_ticket)
print(f"  Agent: {result1['assigned_agent']}")
print(f"  Resolution: {result1['resolution'][:100]}...\n")

# Scenario 2: Simple question -> Normal Agent
simple_ticket = {
    "ticket_id": "TKT-SIMPLE",
    "title": "How do I change my email address?",
    "description": "I want to update my account email",
    "customer_tier": "Standard",
    "category": "", "priority": "", "assigned_agent": "",
    "needs_escalation": False, "processing_steps": [],
    "status": "New", "resolution": ""
}

print("Scenario 2: Simple Account Question")
result2 = conditional_app.invoke(simple_ticket)
print(f"  Agent: {result2['assigned_agent']}")
print(f"  Resolution: {result2['resolution']}")

## Summary

LangGraph provides graph-based workflow orchestration with:
- **State**: Persistent, shared data that evolves through the workflow
- **Nodes**: Functions that process state and return updates
- **Edges**: Fixed or conditional connections between nodes
- **Conditional routing**: Dynamic paths based on state values

### Resources
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [GitHub: langchain-ai/langgraph](https://github.com/langchain-ai/langgraph)
- [LangChain Academy LangGraph Course](https://academy.langchain.com/)